In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates
import matplotlib.dates as mdates

import time
import seaborn as sns

import random
import matplotlib.dates as mdates #dates
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set()

In [2]:
plt.style.use('seaborn')

In [3]:
data = pd.read_csv('jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv')

In [4]:
data.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [7]:
import datetime as dt
data['created_date'] = pd.to_datetime(data['created_date'])

In [8]:
type(data.created_date[1])

pandas._libs.tslibs.timestamps.Timestamp

Trying to fix Unicode Error from the plotly graph below:

In [1]:
#data['created_date_utf8']= str.encode(data['created_date'], 'utf-8')

Create seperate features of hour, day, month, year.

In [ ]:
#extract hour
data['created_hour'] = data['created_date'].dt.hour

In [ ]:
#extract day
data['created_day'] = data['created_date'].dt.day

In [ ]:
#extract month
data['created_month'] = data['created_date'].dt.month

In [ ]:
#extract year
data['created_year'] = data['created_date'].dt.year

In [ ]:
data.head(2)

In [ ]:
data['created_hour'].min()

In [ ]:
data['created_hour'].max()

In [ ]:
tox_by_month = data.groupby('created_hour').mean()['toxic'].reset_index()
tox_by_month.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
data.groupby('created_hour').mean()['toxic'].plot(ax=ax);

In [ ]:
tox_by_year = data.groupby('created_day').mean()['toxic'].reset_index()
tox_by_year.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
data.groupby('created_day').mean()['toxic'].plot(ax=ax);

In [ ]:
tox_by_hour = data.groupby('created_month').mean()['toxic'].reset_index()
tox_by_hour.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
data.groupby('created_month').mean()['toxic'].plot(ax=ax)

In [ ]:
tox_by_day = data.groupby('created_year').mean()['toxic'].reset_index()
tox_by_day.round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
data.groupby('created_year').mean()['toxic'].plot(ax=ax);

Times with highest comment frequency:

In [ ]:
data['created_hour'].mode()

In [ ]:
data['created_day'].mode()

In [ ]:
data['created_month'].mode()

In [ ]:
data['created_year'].mode()

In [ ]:
#earliest post
data['created_date'].min()

In [ ]:
#latest post
data['created_date'].max()

In [ ]:
#tox_hour = pd.to_datetime(data.timestamp)
#toxic_time = pd.Series(data=np.array(data.toxic), 
 #                          index=pd.DatetimeIndex(temperature_time_arr), dtype="float")

In [ ]:
#ts_temperature_hour = ts_temperature.resample("H").mean()

In [ ]:
#ax = plt.figure(figsize=(5,2), dpi=150).add_subplot(111)
#ts_temperature_hour.plot(ax=ax, title="Temperature per hour", color="red")

**Plotly**

In [10]:
import chart_studio
chart_studio.tools.set_credentials_file(username='dren', api_key='••••••••••')

In [11]:
import chart_studio
chart_studio.tools.set_config_file(world_readable=False,
                             sharing='private')

In [12]:
import chart_studio.plotly as py
import plotly.graph_objects as go

In [2]:
time_data = go.Scatter(x= data.created_date,
                         y=data.toxic)

layout = go.Layout(title='Toxic Times', xaxis=dict(title='Date'),
                   yaxis=dict(title='(toxicity of comments)'))

fig = go.Figure(data=[time_data], layout=layout)
#py.iplot(fig, sharing='public')

NameError: name 'go' is not defined